In [2]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from Arch_Unet3D import *
from metrics3D import *
from Datagenerators import *

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_3Dunet(n_channels=3, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 128, 128, 12  1312        ['input_1[0][0]']                
                                8, 16)                                                            
                                                                                                  
 instance_normalization (Instan  (None, 128, 128, 12  32         ['conv3d[0][0]']                 
 ceNormalization)               8, 16)                                                        

                                                                                                  
 activation_6 (Activation)      (None, 16, 16, 16,   0           ['instance_normalization_6[0][0]'
                                128)                             ]                                
                                                                                                  
 dropout_3 (Dropout)            (None, 16, 16, 16,   0           ['activation_6[0][0]']           
                                128)                                                              
                                                                                                  
 conv3d_7 (Conv3D)              (None, 16, 16, 16,   442496      ['dropout_3[0][0]']              
                                128)                                                              
                                                                                                  
 instance_

 tanceNormalization)            64)                                                               
                                                                                                  
 activation_12 (Activation)     (None, 32, 32, 32,   0           ['instance_normalization_12[0][0]
                                64)                              ']                               
                                                                                                  
 dropout_6 (Dropout)            (None, 32, 32, 32,   0           ['activation_12[0][0]']          
                                64)                                                               
                                                                                                  
 conv3d_15 (Conv3D)             (None, 32, 32, 32,   110656      ['dropout_6[0][0]']              
                                64)                                                               
          

In [3]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [4]:
print(len(train_ids))
print(len(val_ids))
print(len(test_ids))

200
60
36


In [5]:
training_generator = DataGenerator3D(train_ids)
valid_generator = DataGenerator3D(val_ids)
test_generator = DataGenerator3D(test_ids)

In [6]:
csv_logger = CSVLogger('Logs/training/3D_U_net_instancenormnodrop.log', separator=',', append=True)

eval_csv_logger = CSVLogger('Logs/testing/3D_U_net_instancenormnodrop.log', separator=',', append=True)

callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0000001, verbose=1),
        csv_logger
    ]

In [7]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/3D/3D_U_net_instancenormfulldrop.h5")

Epoch 1/100
200/200 [==============================] - 129s 563ms/step - loss: 0.1430 - accuracy: 0.9585 - mean_io_u: 0.3750 - dice_coef: 0.3511 - dice_coef_necrotic: 0.0926 - dice_coef_edema: 0.4142 - dice_coef_enhancing: 0.2606 - val_loss: 0.0968 - val_accuracy: 0.9666 - val_mean_io_u: 0.3750 - val_dice_coef: 0.4013 - val_dice_coef_necrotic: 0.1157 - val_dice_coef_edema: 0.5302 - val_dice_coef_enhancing: 0.3414 - lr: 0.0100
Epoch 2/100
200/200 [==============================] - 112s 559ms/step - loss: 0.0792 - accuracy: 0.9723 - mean_io_u: 0.3750 - dice_coef: 0.4374 - dice_coef_necrotic: 0.1547 - dice_coef_edema: 0.5661 - dice_coef_enhancing: 0.4369 - val_loss: 0.0914 - val_accuracy: 0.9721 - val_mean_io_u: 0.3750 - val_dice_coef: 0.5089 - val_dice_coef_necrotic: 0.1038 - val_dice_coef_edema: 0.6221 - val_dice_coef_enhancing: 0.6186 - lr: 0.0100
Epoch 3/100
200/200 [==============================] - 112s 559ms/step - loss: 0.0661 - accuracy: 0.9776 - mean_io_u: 0.3762 - dice_coef: 0.

200/200 [==============================] - 111s 555ms/step - loss: 0.0434 - accuracy: 0.9846 - mean_io_u: 0.3828 - dice_coef: 0.6110 - dice_coef_necrotic: 0.4171 - dice_coef_edema: 0.7469 - dice_coef_enhancing: 0.7029 - val_loss: 0.0496 - val_accuracy: 0.9809 - val_mean_io_u: 0.4002 - val_dice_coef: 0.6155 - val_dice_coef_necrotic: 0.4412 - val_dice_coef_edema: 0.7271 - val_dice_coef_enhancing: 0.7072 - lr: 0.0100
Epoch 21/100
200/200 [==============================] - 111s 556ms/step - loss: 0.0419 - accuracy: 0.9851 - mean_io_u: 0.4063 - dice_coef: 0.6189 - dice_coef_necrotic: 0.4311 - dice_coef_edema: 0.7499 - dice_coef_enhancing: 0.7087 - val_loss: 0.0581 - val_accuracy: 0.9803 - val_mean_io_u: 0.3959 - val_dice_coef: 0.6161 - val_dice_coef_necrotic: 0.3954 - val_dice_coef_edema: 0.7366 - val_dice_coef_enhancing: 0.6804 - lr: 0.0100
Epoch 22/100
200/200 [==============================] - 111s 555ms/step - loss: 0.0425 - accuracy: 0.9847 - mean_io_u: 0.4107 - dice_coef: 0.6145 - dic

Epoch 38/100
200/200 [==============================] - 113s 563ms/step - loss: 0.0296 - accuracy: 0.9891 - mean_io_u: 0.4384 - dice_coef: 0.6935 - dice_coef_necrotic: 0.5725 - dice_coef_edema: 0.8176 - dice_coef_enhancing: 0.7669 - val_loss: 0.0430 - val_accuracy: 0.9853 - val_mean_io_u: 0.4509 - val_dice_coef: 0.6789 - val_dice_coef_necrotic: 0.5284 - val_dice_coef_edema: 0.8061 - val_dice_coef_enhancing: 0.7493 - lr: 4.0000e-04
Epoch 39/100
200/200 [==============================] - 114s 568ms/step - loss: 0.0294 - accuracy: 0.9892 - mean_io_u: 0.4404 - dice_coef: 0.6945 - dice_coef_necrotic: 0.5694 - dice_coef_edema: 0.8190 - dice_coef_enhancing: 0.7707 - val_loss: 0.0432 - val_accuracy: 0.9852 - val_mean_io_u: 0.4540 - val_dice_coef: 0.6791 - val_dice_coef_necrotic: 0.5355 - val_dice_coef_edema: 0.8049 - val_dice_coef_enhancing: 0.7457 - lr: 4.0000e-04
Epoch 40/100
200/200 [==============================] - 112s 561ms/step - loss: 0.0294 - accuracy: 0.9892 - mean_io_u: 0.4397 - di

Epoch 55/100
200/200 [==============================] - 114s 570ms/step - loss: 0.0287 - accuracy: 0.9895 - mean_io_u: 0.4449 - dice_coef: 0.6998 - dice_coef_necrotic: 0.5820 - dice_coef_edema: 0.8226 - dice_coef_enhancing: 0.7725 - val_loss: 0.0437 - val_accuracy: 0.9853 - val_mean_io_u: 0.4569 - val_dice_coef: 0.6823 - val_dice_coef_necrotic: 0.5357 - val_dice_coef_edema: 0.8043 - val_dice_coef_enhancing: 0.7497 - lr: 3.2000e-06
Epoch 56/100
200/200 [==============================] - ETA: 0s - loss: 0.0287 - accuracy: 0.9894 - mean_io_u: 0.4451 - dice_coef: 0.6992 - dice_coef_necrotic: 0.5805 - dice_coef_edema: 0.8226 - dice_coef_enhancing: 0.7712
Epoch 56: ReduceLROnPlateau reducing learning rate to 6.399999165296323e-07.
200/200 [==============================] - 114s 571ms/step - loss: 0.0287 - accuracy: 0.9894 - mean_io_u: 0.4451 - dice_coef: 0.6992 - dice_coef_necrotic: 0.5805 - dice_coef_edema: 0.8226 - dice_coef_enhancing: 0.7712 - val_loss: 0.0437 - val_accuracy: 0.9853 - val

Epoch 72/100
200/200 [==============================] - 112s 561ms/step - loss: 0.0287 - accuracy: 0.9894 - mean_io_u: 0.4446 - dice_coef: 0.6992 - dice_coef_necrotic: 0.5791 - dice_coef_edema: 0.8217 - dice_coef_enhancing: 0.7718 - val_loss: 0.0437 - val_accuracy: 0.9853 - val_mean_io_u: 0.4569 - val_dice_coef: 0.6823 - val_dice_coef_necrotic: 0.5357 - val_dice_coef_edema: 0.8043 - val_dice_coef_enhancing: 0.7497 - lr: 1.0000e-07
Epoch 73/100
200/200 [==============================] - 112s 559ms/step - loss: 0.0287 - accuracy: 0.9894 - mean_io_u: 0.4449 - dice_coef: 0.6995 - dice_coef_necrotic: 0.5797 - dice_coef_edema: 0.8224 - dice_coef_enhancing: 0.7723 - val_loss: 0.0437 - val_accuracy: 0.9853 - val_mean_io_u: 0.4569 - val_dice_coef: 0.6823 - val_dice_coef_necrotic: 0.5357 - val_dice_coef_edema: 0.8043 - val_dice_coef_enhancing: 0.7497 - lr: 1.0000e-07
Epoch 74/100
200/200 [==============================] - 111s 555ms/step - loss: 0.0287 - accuracy: 0.9894 - mean_io_u: 0.4450 - di

Epoch 91/100
200/200 [==============================] - 117s 586ms/step - loss: 0.0286 - accuracy: 0.9895 - mean_io_u: 0.4452 - dice_coef: 0.7001 - dice_coef_necrotic: 0.5822 - dice_coef_edema: 0.8229 - dice_coef_enhancing: 0.7729 - val_loss: 0.0437 - val_accuracy: 0.9853 - val_mean_io_u: 0.4569 - val_dice_coef: 0.6823 - val_dice_coef_necrotic: 0.5357 - val_dice_coef_edema: 0.8043 - val_dice_coef_enhancing: 0.7497 - lr: 1.0000e-07
Epoch 92/100
200/200 [==============================] - 117s 582ms/step - loss: 0.0287 - accuracy: 0.9894 - mean_io_u: 0.4454 - dice_coef: 0.6995 - dice_coef_necrotic: 0.5811 - dice_coef_edema: 0.8227 - dice_coef_enhancing: 0.7727 - val_loss: 0.0437 - val_accuracy: 0.9853 - val_mean_io_u: 0.4569 - val_dice_coef: 0.6823 - val_dice_coef_necrotic: 0.5357 - val_dice_coef_edema: 0.8043 - val_dice_coef_enhancing: 0.7497 - lr: 1.0000e-07
Epoch 93/100
200/200 [==============================] - 117s 585ms/step - loss: 0.0287 - accuracy: 0.9894 - mean_io_u: 0.4449 - di

In [7]:
model = keras.models.load_model('Models/3D/3D_U_net_instancenormnodrop.h5', 
                                   custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
                                                   "dice_coef": dice_coef,
                                                   "dice_coef_necrotic": dice_coef_necrotic,
                                                   "dice_coef_edema": dice_coef_edema,
                                                   "dice_coef_enhancing": dice_coef_enhancing
                                                  }, compile=True)

In [8]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

36/36 [==============================] - 23s 328ms/step - loss: 0.0506 - accuracy: 0.9862 - mean_io_u: 0.4054 - dice_coef: 0.7484 - dice_coef_necrotic: 0.6348 - dice_coef_edema: 0.8197 - dice_coef_enhancing: 0.8383


[0.050608403980731964,
 0.9861829280853271,
 0.40539684891700745,
 0.7483542561531067,
 0.6347764134407043,
 0.8196970224380493,
 0.8383021354675293]

In [9]:
valid_csv_logger = CSVLogger('Logs/validation/3D_U_net_instancenormfulldrop.log', separator=',', append=True)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)

60/60 [==============================] - 20s 334ms/step - loss: 0.0437 - accuracy: 0.9853 - mean_io_u: 0.4570 - dice_coef: 0.6823 - dice_coef_necrotic: 0.5357 - dice_coef_edema: 0.8043 - dice_coef_enhancing: 0.7497


[0.04366374760866165,
 0.9852508902549744,
 0.4569525122642517,
 0.6822844743728638,
 0.5357131361961365,
 0.8043010830879211,
 0.7496659159660339]